In [1]:
import joblib

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV
from sklearn.svm import SVC
import sklearn.metrics
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [3]:
symptoms_track = ['Nausea', 'Headache','diarrhea','Soar Throat', 'Fever', 'Muscle Ache', 'Loss of Smell or Taste', 'Cough' , 'Shortness of Breath', 'Feeling tired']

In [28]:
symp_dis_df = pd.read_csv('symp_dis.csv')
symp_dis_df = shuffle(symp_dis_df,random_state=42)
symp_dis_df.head()

,User,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
373,U374,Acne,skin_rash,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,U4917,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1550,U1551,Hyperthyroidism,fatigue,mood_swings,weight_loss,restlessness,sweating,diarrhoea,fast_heart_rate,excessive_hunger,muscle_weakness,irritability,abnormal_menstruation,NaN,NaN,NaN,NaN,NaN,NaN
3081,U3082,AIDS,muscle_wasting,patches_in_throat,high_fever,extra_marital_contacts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3857,U3858,Chronic cholestasis,itching,vomiting,yellowish_skin,nausea,loss_of_appetite,abdominal_pain,yellowing_of_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
for col in symp_dis_df.columns:

    symp_dis_df[col] = symp_dis_df[col].str.replace('_',' ')
symp_dis_df.head()

,User,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
373,U374,Acne,skin rash,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,U4917,Acne,skin rash,pus filled pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1550,U1551,Hyperthyroidism,fatigue,mood swings,weight loss,restlessness,sweating,diarrhoea,fast heart rate,excessive hunger,muscle weakness,irritability,abnormal menstruation,NaN,NaN,NaN,NaN,NaN,NaN
3081,U3082,AIDS,muscle wasting,patches in throat,high fever,extra marital contacts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3857,U3858,Chronic cholestasis,itching,vomiting,yellowish skin,nausea,loss of appetite,abdominal pain,yellowing of eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
symp_dis_df.describe()

,User,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
count,4920,4920,4920,4920,4920,4572,3714,2934,2268,1944,1692,1512,1194,744,504,306,240,192,72
unique,4920,41,34,48,54,50,38,32,26,21,22,21,18,11,8,4,3,3,1
top,U374,Acne,vomiting,vomiting,fatigue,high fever,headache,nausea,abdominal pain,abdominal pain,yellowing of eyes,yellowing of eyes,irritability,malaise,stomach bleeding,chest pain,chest pain,loss of smell,muscle pain
freq,1,120,822,870,726,378,348,390,264,276,228,198,120,126,72,96,144,72,72


In [31]:
null_checker = symp_dis_df.apply(lambda x: sum(x.isnull())).to_frame(name='count')
print(null_checker)

            count
User            0
Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Symptom_4     348
Symptom_5    1206
Symptom_6    1986
Symptom_7    2652
Symptom_8    2976
Symptom_9    3228
Symptom_10   3408
Symptom_11   3726
Symptom_12   4176
Symptom_13   4416
Symptom_14   4614
Symptom_15   4680
Symptom_16   4728
Symptom_17   4848


In [32]:
symp_dis_df.describe()

,User,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
count,4920,4920,4920,4920,4920,4572,3714,2934,2268,1944,1692,1512,1194,744,504,306,240,192,72
unique,4920,41,34,48,54,50,38,32,26,21,22,21,18,11,8,4,3,3,1
top,U374,Acne,vomiting,vomiting,fatigue,high fever,headache,nausea,abdominal pain,abdominal pain,yellowing of eyes,yellowing of eyes,irritability,malaise,stomach bleeding,chest pain,chest pain,loss of smell,muscle pain
freq,1,120,822,870,726,378,348,390,264,276,228,198,120,126,72,96,144,72,72


In [33]:
cols = symp_dis_df.columns
data = symp_dis_df[cols].values.flatten()

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(symp_dis_df.shape)

symp_dis_df = pd.DataFrame(s, columns=symp_dis_df.columns)
symp_dis_df.head()

,User,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,U374,Acne,skin rash,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U4917,Acne,skin rash,pus filled pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U1551,Hyperthyroidism,fatigue,mood swings,weight loss,restlessness,sweating,diarrhoea,fast heart rate,excessive hunger,muscle weakness,irritability,abnormal menstruation,NaN,NaN,NaN,NaN,NaN,NaN
3,U3082,AIDS,muscle wasting,patches in throat,high fever,extra marital contacts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U3858,Chronic cholestasis,itching,vomiting,yellowish skin,nausea,loss of appetite,abdominal pain,yellowing of eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
symp_dis_df = symp_dis_df.fillna(0)
symp_dis_df.head()

,User,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,U374,Acne,skin rash,blackheads,scurring,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,U4917,Acne,skin rash,pus filled pimples,blackheads,scurring,0,0,0,0,0,0,0,0,0,0,0,0,0
2,U1551,Hyperthyroidism,fatigue,mood swings,weight loss,restlessness,sweating,diarrhoea,fast heart rate,excessive hunger,muscle weakness,irritability,abnormal menstruation,0,0,0,0,0,0
3,U3082,AIDS,muscle wasting,patches in throat,high fever,extra marital contacts,0,0,0,0,0,0,0,0,0,0,0,0,0
4,U3858,Chronic cholestasis,itching,vomiting,yellowish skin,nausea,loss of appetite,abdominal pain,yellowing of eyes,0,0,0,0,0,0,0,0,0,0


In [35]:
symp_dis_df.values

array([['U374', 'Acne', 'skin rash', ..., 0, 0, 0],
       ['U4917', 'Acne', 'skin rash', ..., 0, 0, 0],
       ['U1551', 'Hyperthyroidism', 'fatigue', ..., 0, 0, 0],
       ...,
       ['U3093', 'Dengue', 'skin rash', ..., 0, 0, 0],
       ['U3773', 'Fungal infection', 'itching', ..., 0, 0, 0],
       ['U861', 'Drug Reaction', 'itching', ..., 0, 0, 0]], dtype=object)

In [36]:
symptom_severity_df = pd.read_csv('Symptom-severity.csv')
symptom_severity_df['Symptom'] = symptom_severity_df['Symptom'].str.replace('_',' ')
symptom_severity_df.head()

,Symptom,weight
0,itching,1
1,skin rash,3
2,nodal skin eruptions,4
3,continuous sneezing,4
4,shivering,5


In [37]:
symptom_severity_df['Symptom'].unique()

array(['itching', 'skin rash', 'nodal skin eruptions',
       'continuous sneezing', 'shivering', 'chills', 'joint pain',
       'stomach pain', 'acidity', 'ulcers on tongue', 'muscle wasting',
       'vomiting', 'burning micturition', 'spotting urination', 'fatigue',
       'weight gain', 'anxiety', 'cold hands and feets', 'mood swings',
       'weight loss', 'restlessness', 'lethargy', 'patches in throat',
       'irregular sugar level', 'cough', 'high fever', 'sunken eyes',
       'breathlessness', 'sweating', 'dehydration', 'indigestion',
       'headache', 'yellowish skin', 'dark urine', 'nausea',
       'loss of appetite', 'pain behind the eyes', 'back pain',
       'constipation', 'abdominal pain', 'diarrhoea', 'mild fever',
       'yellow urine', 'yellowing of eyes', 'acute liver failure',
       'fluid overload', 'swelling of stomach', 'swelled lymph nodes',
       'malaise', 'blurred and distorted vision', 'phlegm',
       'throat irritation', 'redness of eyes', 'sinus pressu

In [38]:
vals = symp_dis_df.values
symptoms = symptom_severity_df['Symptom'].unique()

for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = symptom_severity_df[symptom_severity_df['Symptom'] == symptoms[i]]['weight'].values[0]

d = pd.DataFrame(vals, columns=cols)
d.head()

,User,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,U374,Acne,3,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,U4917,Acne,3,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
2,U1551,Hyperthyroidism,4,3,3,5,3,6,5,4,2,2,6,0,0,0,0,0,0
3,U3082,AIDS,3,6,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0
4,U3858,Chronic cholestasis,1,5,3,5,4,4,4,0,0,0,0,0,0,0,0,0,0


In [53]:
d = d.replace('dischromic  patches', 0)
d = d.replace('spotting  urination',0)
symp_dis_df = d.replace('foul smell of urine',0)
symp_dis_df.head(10)

,User,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,U374,Acne,3,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,U4917,Acne,3,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
2,U1551,Hyperthyroidism,4,3,3,5,3,6,5,4,2,2,6,0,0,0,0,0,0
3,U3082,AIDS,3,6,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0
4,U3858,Chronic cholestasis,1,5,3,5,4,4,4,0,0,0,0,0,0,0,0,0,0
5,U927,Hypertension,3,7,4,4,3,0,0,0,0,0,0,0,0,0,0,0,0
6,U2576,Hypoglycemia,5,4,4,3,3,5,5,4,4,4,2,4,0,0,0,0,0
7,U766,Arthritis,2,4,5,2,0,0,0,0,0,0,0,0,0,0,0,0,0
8,U3752,Hepatitis B,1,4,2,3,4,4,4,4,4,6,5,2,0,0,0,0,0
9,U4194,Migraine,3,5,3,5,4,4,3,2,3,0,0,0,0,0,0,0,0


In [54]:
null_checker = symp_dis_df.apply(lambda x: sum(x.isnull())).to_frame(name='count')
print(null_checker)

            count
User            0
Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Symptom_4       0
Symptom_5       0
Symptom_6       0
Symptom_7       0
Symptom_8       0
Symptom_9       0
Symptom_10      0
Symptom_11      0
Symptom_12      0
Symptom_13      0
Symptom_14      0
Symptom_15      0
Symptom_16      0
Symptom_17      0


In [55]:
print("Number of symptoms used to identify the disease ",len(symptom_severity_df['Symptom'].unique()))
print("Number of diseases that can be identified ",len(symp_dis_df['Disease'].unique()))

Number of symptoms used to identify the disease  132
Number of diseases that can be identified  41


In [56]:
symp_dis_df['Disease'].unique()

array(['Acne', 'Hyperthyroidism', 'AIDS', 'Chronic cholestasis',
       'Hypertension', 'Hypoglycemia', 'Arthritis', 'Hepatitis B',
       'Migraine', 'Urinary tract infection', 'Diabetes', 'Hepatitis D',
       'Psoriasis', 'Alcoholic hepatitis', 'Dimorphic hemmorhoids(piles)',
       'Hepatitis E', 'Cervical spondylosis', 'Bronchial Asthma',
       'hepatitis A', 'Allergy', 'Hepatitis C', 'Pneumonia',
       'Hypothyroidism', 'Gastroenteritis', 'Varicose veins', 'Jaundice',
       'Drug Reaction', '(vertigo) Paroymsal  Positional Vertigo',
       'Heart attack', 'Tuberculosis', 'Typhoid', 'Common Cold',
       'Peptic ulcer diseae', 'Paralysis (brain hemorrhage)',
       'Fungal infection', 'Impetigo', 'GERD', 'Dengue', 'Malaria',
       'Chicken pox', 'Osteoarthristis'], dtype=object)

In [57]:
# Setting the 'User' column as the index
symp_dis_df.set_index('User', inplace=True)

In [58]:
symp_dis_df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
User,,,,,,,,,,,,,,,,,,
U374,Acne,3,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
U4917,Acne,3,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
U1551,Hyperthyroidism,4,3,3,5,3,6,5,4,2,2,6,0,0,0,0,0,0
U3082,AIDS,3,6,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0
U3858,Chronic cholestasis,1,5,3,5,4,4,4,0,0,0,0,0,0,0,0,0,0


In [59]:
data = symp_dis_df.iloc[:,1:].values
labels = symp_dis_df['Disease'].values

In [60]:
from sklearn.feature_selection import RFE
rf1 = RandomForestClassifier()

# Initialize RFE with the Random Forest classifier and desired number of features
rfe = RFE(estimator=rf1, n_features_to_select=10)
rfe.fit(data, labels)
# Get the support mask indicating selected features
selected_feature_indices = rfe.support_

# Get the column names of selected features from the original DataFrame
selected_features = symp_dis_df.columns[1:][selected_feature_indices]
print(selected_features)
data = data[:,selected_feature_indices ]
print(data)

Index(['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5',
       'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10'],
      dtype='object')
[[3 2 2 ... 0 0 0]
 [3 2 2 ... 0 0 0]
 [4 3 3 ... 4 2 2]
 ...
 [3 3 3 ... 5 4 4]
 [1 3 4 ... 0 0 0]
 [1 3 5 ... 0 0 0]]


In [61]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size = 0.8,random_state=10)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(3936, 10) (984, 10) (3936,) (984,)


In [62]:
print(data.shape)

(4920, 10)


In [63]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(6,5),
                    random_state=10,
                    verbose=True,
                    learning_rate_init=0.01,
                    max_iter=500)
clf.fit(x_train, y_train)

Iteration 1, loss = 3.68809738
Iteration 2, loss = 3.42764760
Iteration 3, loss = 3.04290952
Iteration 4, loss = 2.62416456
Iteration 5, loss = 2.27639746
Iteration 6, loss = 1.99475249
Iteration 7, loss = 1.81060772
Iteration 8, loss = 1.67310392
Iteration 9, loss = 1.57674389
Iteration 10, loss = 1.50123856
Iteration 11, loss = 1.43899037
Iteration 12, loss = 1.37100446
Iteration 13, loss = 1.31820313
Iteration 14, loss = 1.28550680
Iteration 15, loss = 1.25173820
Iteration 16, loss = 1.19559989
Iteration 17, loss = 1.14836327
Iteration 18, loss = 1.10990476
Iteration 19, loss = 1.06773205
Iteration 20, loss = 1.02915175
Iteration 21, loss = 1.01431823
Iteration 22, loss = 0.96245279
Iteration 23, loss = 0.92575118
Iteration 24, loss = 0.90037588
Iteration 25, loss = 0.87313003
Iteration 26, loss = 0.85839921
Iteration 27, loss = 0.85708080
Iteration 28, loss = 0.80499570
Iteration 29, loss = 0.78563647
Iteration 30, loss = 0.76638024
Iteration 31, loss = 0.74528865
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(6, 5), learning_rate_init=0.01, max_iter=500,
              random_state=10, verbose=True)

In [64]:
# Make prediction on test dataset
ypred=clf.predict(x_test)

# Import accuracy score
from sklearn.metrics import accuracy_score, f1_score

# Calcuate accuracy
accuracy_score(y_test,ypred)
print('F1-score% =', f1_score(y_test, ypred, average='macro')*100, '|', 'Accuracy% =', accuracy_score(y_test, ypred)*100)

F1-score% = 92.85032453245658 | Accuracy% = 92.78455284552845


In [65]:
x_test.shape

(984, 10)

In [66]:
import joblib

# Save the model to a file
joblib.dump(clf, 'mlp_classifier.joblib')

['mlp_classifier.joblib']

In [67]:
symptom_severity_df

,Symptom,weight
0,itching,1
1,skin rash,3
2,nodal skin eruptions,4
3,continuous sneezing,4
4,shivering,5
...,...,...
128,inflammatory nails,2
129,blister,4
130,red sore around nose,2
131,yellow crust ooze,3


In [68]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from scipy.spatial.distance import cosine

# Load ClinicalBERT
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    input_mask_expanded = inputs['attention_mask'].unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
    sum_embeddings = torch.sum(outputs.last_hidden_state * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def calculate_similarity(embedding1, embedding2):
  # Flatten the embeddings to 1-D
    embedding1_flat = embedding1.flatten()
    embedding2_flat = embedding2.flatten()

    return 1 - cosine(embedding1_flat, embedding2_flat)

symps = ['diarrhea', 'Fever', 'Headache','migrain', 'anxeity', 'Muscle Ache']
# Process for finding similarities
actual_symps = []
actual_symps_sev = []

for s in symps:
    max_sim = 0
    max_sim_idx = 0
    s_embedding = get_embedding(s)

    for i in range(len(symptom_severity_df)):
        symptom_severity_df_symptom_embedding = get_embedding(symptom_severity_df.at[i, 'Symptom'])
        score = calculate_similarity(s_embedding, symptom_severity_df_symptom_embedding)

        if max_sim < score:
            max_sim = score
            max_sim_idx = i

    actual_symps.append(symptom_severity_df.at[max_sim_idx, 'Symptom'])
    actual_symps_sev.append(symptom_severity_df.at[max_sim_idx, 'weight'])

print("Actual Symptoms: ", actual_symps)
print("Actual Symptom Severity: ", actual_symps_sev)

Actual Symptoms:  ['diarrhoea', 'high fever', 'headache', 'polyuria', 'acidity', 'muscle pain']
Actual Symptom Severity:  [6, 7, 3, 4, 3, 2]


In [69]:
target_size = 10

# Extend the list with zeros until it reaches the target size
while len(actual_symps_sev) < target_size:
    actual_symps_sev.append(0)

# random.shuffle(actual_symps_sev)
print("Actual Symptom weight: ", actual_symps_sev)
test_sample = []
test_sample.append(actual_symps_sev)
clf = joblib.load('mlp_classifier.joblib')
ypred_test=clf.predict(test_sample)
print(ypred_test)

Actual Symptom weight:  [6, 7, 3, 4, 3, 2, 0, 0, 0, 0]
['(vertigo) Paroymsal  Positional Vertigo']


In [70]:
disease_food = pd.read_csv('dataset.csv')

In [71]:
disease_food.head()

,Meal_Id,Name,catagory,description,Veg_Non,Nutrient,Disease,Diet,Price
0,meal_id1,summer squash salad,salad,"white balsamic vinegar, lemon juice, lemon rin...",veg,fiber,obesity diabeties hypertension goitre,alkaline_diet low_fat_diet ketogenic_diet low...,485
1,meal_id2,chicken minced salad,salad,"olive oil, chicken mince, garlic (minced), oni...",non-veg,fiber,anemia pregnancy hypertension rickets goitre ...,low_fat_diet low_carb_diet ketogenic_diet low...,600
2,meal_id3,sweet chilli almonds,chilli,"almonds whole, egg white, curry leaves, salt, ...",veg,vitamin_a,hypertension scurvy heart_disease goitre kidn...,alkaline_diet low_fat_diet paleo_diet Mediter...,255
3,meal_id4,tricolour salad,salad,"vinegar, honey/sugar, soy sauce, salt, garlic ...",veg,fiber,obesity goitre hypertension,low_fat_diet ketogenic_diet low_sodium_diet h...,615
4,meal_id5,gluten-free christmas cake,cake,"christmas dry fruits (pre-soaked), orange zest...",veg,vitamin_a,goitre kidney_disease,high_protien_diet,465


In [72]:
disease1 = str(ypred_test[0])
print(type(disease1))
print(type(disease_food.at[0, 'Disease']))
# Process for finding similarities

max_sim = 0
max_sim_idx = 0

df = pd.DataFrame()
df = disease_food
simScoreList = []

for i in range(len(disease_food)):
  disease_food_tab = get_embedding(disease_food.at[i, 'Disease'])
  # print(type(disease1))
  disease_embedding = get_embedding(disease1)
  score = calculate_similarity(disease_embedding, disease_food_tab)
  simScoreList.append(score)

df['SimScore'] = simScoreList

print("Actual Disease: ", disease1)
df

<class 'str'>
<class 'str'>
Actual Disease:  (vertigo) Paroymsal  Positional Vertigo


,Meal_Id,Name,catagory,description,Veg_Non,Nutrient,Disease,Diet,Price,SimScore
0,meal_id1,summer squash salad,salad,"white balsamic vinegar, lemon juice, lemon rin...",veg,fiber,obesity diabeties hypertension goitre,alkaline_diet low_fat_diet ketogenic_diet low...,485,0.839766
1,meal_id2,chicken minced salad,salad,"olive oil, chicken mince, garlic (minced), oni...",non-veg,fiber,anemia pregnancy hypertension rickets goitre ...,low_fat_diet low_carb_diet ketogenic_diet low...,600,0.851694
2,meal_id3,sweet chilli almonds,chilli,"almonds whole, egg white, curry leaves, salt, ...",veg,vitamin_a,hypertension scurvy heart_disease goitre kidn...,alkaline_diet low_fat_diet paleo_diet Mediter...,255,0.883409
3,meal_id4,tricolour salad,salad,"vinegar, honey/sugar, soy sauce, salt, garlic ...",veg,fiber,obesity goitre hypertension,low_fat_diet ketogenic_diet low_sodium_diet h...,615,0.841578
4,meal_id5,gluten-free christmas cake,cake,"christmas dry fruits (pre-soaked), orange zest...",veg,vitamin_a,goitre kidney_disease,high_protien_diet,465,0.859329
...,...,...,...,...,...,...,...,...,...,...
374,meal_id299,kolim / jawla,fish,dried fish named kolim or jawla found in coast...,veg,vitamin_a,cancer scurvy heart_disease eye_disease ricke...,alkaline_diet low_fat_diet ketogenic_diet veg...,565,0.852202
375,meal_id306,banana chips,banana,dried slices of bananas (fruits of herbaceous ...,veg,magnesium,hypertension,alkaline_diet high_protien_diet vegan_diet hi...,295,0.807233
376,meal_id307,bhurji- egg,egg,"made using indian spices, onion, tomatoes, gre...",non-veg,vitamin_a,hypertension rickets goitre kidney_disease pr...,low_sodium_diet high_protien_diet low_fat_die...,645,0.857216
377,meal_id308,flattened rice / pohe,rice,"edible, dehusked rice which is flattened into ...",veg,carbohydrates,anemia cancer scurvy heart_disease eye_diseas...,alkaline_diet low_fat_diet ketogenic_diet veg...,525,0.856874


In [73]:
df.nlargest(3, 'SimScore')

,Meal_Id,Name,catagory,description,Veg_Non,Nutrient,Disease,Diet,Price,SimScore
99,meal_id95,steamed chicken roulade,chicken,"lamb mince, garlic , salt, paprika powder, pom...",non-veg,iron,anemia scurvy rickets goitre kidney_disease p...,low_fat_diet low_carb_diet vegan_diet high_fi...,235,0.886461
227,meal_id225,sweet potato pie,pie,"yams (red skinned), condensed milk, sugar, egg...",veg,vitamin_a,hypertension scurvy goitre kidney_disease pre...,alkaline_diet low_fat_diet paleo_diet Mediter...,680,0.883550
258,meal_id225,sweet potato pie,pie,"yams (red skinned), condensed milk, sugar, egg...",veg,vitamin_a,hypertension scurvy goitre kidney_disease pre...,alkaline_diet low_fat_diet paleo_diet Mediter...,415,0.883550


In [74]:
df.loc[df['Veg_Non'] == 'veg'].nlargest(3, 'SimScore')

,Meal_Id,Name,catagory,description,Veg_Non,Nutrient,Disease,Diet,Price,SimScore
227,meal_id225,sweet potato pie,pie,"yams (red skinned), condensed milk, sugar, egg...",veg,vitamin_a,hypertension scurvy goitre kidney_disease pre...,alkaline_diet low_fat_diet paleo_diet Mediter...,680,0.883550
258,meal_id225,sweet potato pie,pie,"yams (red skinned), condensed milk, sugar, egg...",veg,vitamin_a,hypertension scurvy goitre kidney_disease pre...,alkaline_diet low_fat_diet paleo_diet Mediter...,415,0.883550
2,meal_id3,sweet chilli almonds,chilli,"almonds whole, egg white, curry leaves, salt, ...",veg,vitamin_a,hypertension scurvy heart_disease goitre kidn...,alkaline_diet low_fat_diet paleo_diet Mediter...,255,0.883409


In [75]:
df.loc[df['Veg_Non'] == 'non-veg'].nlargest(3, 'SimScore')

,Meal_Id,Name,catagory,description,Veg_Non,Nutrient,Disease,Diet,Price,SimScore
99,meal_id95,steamed chicken roulade,chicken,"lamb mince, garlic , salt, paprika powder, pom...",non-veg,iron,anemia scurvy rickets goitre kidney_disease p...,low_fat_diet low_carb_diet vegan_diet high_fi...,235,0.886461
116,meal_id111,plum and cherry roasted chicken,chicken,"carrot (finely diced), onion (finely diced), c...",non-veg,iron,anemia hypertension scurvy rickets goitre kid...,alkaline_diet low_fat_diet low_carb_diet vega...,335,0.876979
38,meal_id48,almond and chicken momos (without shell),snacks,"chicken mince, garlic, carrots, spring onion, ...",non-veg,calcium,anemia hypertension rickets goitre kidney_dis...,low_fat_diet low_carb_diet ketogenic_diet low...,415,0.870556


In [79]:
def recommend_diet(age, weight, heart_rate):
    # Define threshold ranges
    age_groups = {'young': [0, 30], 'middle': [31, 60], 'senior': [61, float('inf')]}
    weight_categories = {'underweight': [0, 50], 'normal': [51, 80], 'overweight': [81, 100], 'obese': [101, float('inf')]}
    heart_rate_zones = {'bradycardia': [0, 60], 'normal': [61, 100], 'elevated': [101, 140], 'tachycardia': [141, float('inf')]}

    # Multi-conditional logic for diet recommendation
    diets = []
    
    # Age-based conditions
    if age_groups['young'][0] <= age <= age_groups['young'][1]:
        if weight <= weight_categories['underweight'][1]:
            diets += ['high_calorie_diet', 'high_protein_diet']
        elif weight <= weight_categories['normal'][1]:
            diets += ['balanced_omni_diet', 'paleo_diet']
        else:
            diets += ['low_carb_diet', 'ketogenic_diet']

    elif age_groups['middle'][0] <= age <= age_groups['middle'][1]:
        if weight_categories['normal'][0] <= weight <= weight_categories['normal'][1]:
            diets += ['vegan_diet', 'type_a_diet']
        elif weight <= weight_categories['overweight'][1]:
            diets += ['Mediterranean_diet']
        else:
            diets += ['low_fat_diet']

    elif age_groups['senior'][0] <= age:
        if weight > weight_categories['obese'][0]:
            diets += ['low_carb_diet']
        else:
            diets += ['Mediterranean_diet', 'alkaline_diet']

    # Heart rate-based conditions
    if heart_rate <= heart_rate_zones['bradycardia'][1]:
        diets += ['high_fiber_diet']
    elif heart_rate <= heart_rate_zones['normal'][1]:
        diets += ['omni_diet']
    elif heart_rate <= heart_rate_zones['elevated'][1]:
        diets += ['DASH_diet', 'low_sodium_diet']
    elif heart_rate > heart_rate_zones['tachycardia'][0]:
        diets += ['DASH_diet']

    # Remove duplicates and return the list of recommended diets
    return list(set(diets))

# Example usage
user_age = 45
user_weight = 75
user_heart_rate = 85

diet_recommendations = recommend_diet(user_age, user_weight, user_heart_rate)
print(f'Recommended diets: {diet_recommendations}')


Recommended diets: ['vegan_diet', 'type_a_diet', 'omni_diet']


In [82]:
filtered_df = df[df['Diet'].apply(lambda diets: any(diet in diets for diet in diet_recommendations))].nlargest(3, 'SimScore')


In [83]:
filtered_df.head()

,Meal_Id,Name,catagory,description,Veg_Non,Nutrient,Disease,Diet,Price,SimScore
99,meal_id95,steamed chicken roulade,chicken,"lamb mince, garlic , salt, paprika powder, pom...",non-veg,iron,anemia scurvy rickets goitre kidney_disease p...,low_fat_diet low_carb_diet vegan_diet high_fi...,235,0.886461
271,meal_id254,orange quinoa sevaiyan,almond,"sevaiyan, quinoa, orange juice , dried figs, s...",veg,protien,anemia scurvy rickets goitre pregnancy,low_sodium_diet vegan_diet low_fat_diet dash_...,480,0.878678
116,meal_id111,plum and cherry roasted chicken,chicken,"carrot (finely diced), onion (finely diced), c...",non-veg,iron,anemia hypertension scurvy rickets goitre kid...,alkaline_diet low_fat_diet low_carb_diet vega...,335,0.876979
